In [ ]:
# @title Colab/Local Setup (core deps)
# Torch is preinstalled on Colab; install extras quietly.
# Install btrack without resolver to avoid conflicts, then compatible deps.
!pip install -q --no-deps btrack==0.6.5
!pip install -q numpy==1.23.5 numba==0.57.1 scipy==1.10.1 dill==0.3.7     albumentations==1.4.14 opencv-python-headless==4.10.0.84     tifffile==2024.8.30 scikit-image==0.24.0 scikit-learn==1.5.2     pandas==2.2.2 tqdm==4.66.4


In [ ]:
# @title Imports & Globals
import os
import math
import shutil
from pathlib import Path
from typing import List, Tuple

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import ConcatDataset, DataLoader, Dataset, Subset
from sklearn.model_selection import KFold
from skimage.measure import label, regionprops
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

# ROI constants
ROI_Y_MIN, ROI_Y_MAX = 512, 768
ROI_X_MIN, ROI_X_MAX = 256, 512
ROI_H, ROI_W = ROI_Y_MAX - ROI_Y_MIN, ROI_X_MAX - ROI_X_MIN

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.backends.cudnn.benchmark = True
print(f"Using device: {device}")


In [ ]:
# @title Data Download Helpers (validation + optional Drive training)
import os, shutil, zipfile, requests

# Download validation video+labels zip (with SSL chain)
def download_validation_data(target_dir: str = "val_data",
                             url: str = "https://su2.utia.cas.cz/files/labs/final2025/val_and_sota.zip",
                             cert_url: str = "https://pki.cesnet.cz/_media/certs/chain-harica-rsa-ov-crosssigned-root.pem"):
    if os.path.exists(target_dir) and len(os.listdir(target_dir)) > 0:
        print(f"'{target_dir}' already exists. Skipping download.")
        return

    chain_path = "chain-harica-cross.pem"
    print("1) Downloading SSL certificate chain...")
    r = requests.get(cert_url, timeout=10, stream=True)
    r.raise_for_status()
    with open(chain_path, "wb") as f:
        f.write(r.content)
    print("2) Downloading validation archive...")
    zip_name = os.path.basename(url)
    with requests.get(url, stream=True, verify=chain_path, timeout=30) as resp:
        resp.raise_for_status()
        with open(zip_name, "wb") as f:
            for chunk in resp.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
    print("3) Extracting...")
    os.makedirs(target_dir, exist_ok=True)
    with zipfile.ZipFile(zip_name, "r") as zf:
        zf.extractall(target_dir)
    os.remove(zip_name)
    print(f"Done. Data in '{target_dir}/'")


# Optional: copy training data from Google Drive (manual annotations)
def fetch_from_drive(DRIVE_SOURCE_PATH: str = "/content/drive/MyDrive/unet_train",
                     TARGET_DIR: str = "real_training_data"):
    try:
        from google.colab import drive
    except ImportError:
        print("google.colab not available (not running in Colab).")
        return

    print("?? Mounting Drive...")
    drive.mount('/content/drive')

    if os.path.exists(TARGET_DIR) and len(os.listdir(TARGET_DIR)) > 0:
        print(f"'{TARGET_DIR}' already exists and is not empty. Skipping copy.")
        return

    if not os.path.exists(DRIVE_SOURCE_PATH):
        print(f"? Source not found: {DRIVE_SOURCE_PATH}")
        return

    print(f"Copying {DRIVE_SOURCE_PATH} -> {TARGET_DIR}")
    try:
        shutil.copytree(DRIVE_SOURCE_PATH, TARGET_DIR)
        print("? Copy complete.")
    except Exception as e:
        print(f"? Copy failed: {e}")
        if os.path.exists(TARGET_DIR):
            shutil.rmtree(TARGET_DIR)


In [ ]:
# @title Model & Loss (Lightweight U-Net)
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate=0.0):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout2d(dropout_rate) if dropout_rate > 0 else None

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        if self.dropout:
            x = self.dropout(x)
        return x

class LightweightUNet(nn.Module):
    def __init__(self, in_channels=1, n_classes=1, features=[16, 32, 64, 128]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        in_ch = in_channels
        for feature in features:
            self.downs.append(ConvBlock(in_ch, feature))
            in_ch = feature

        self.bottleneck = ConvBlock(features[-1], features[-1] * 2)

        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2))
            self.ups.append(ConvBlock(feature * 2, feature))

        self.final_conv = nn.Conv2d(features[0], n_classes, kernel_size=1)

    def forward(self, x):
        skip_connections = []
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx // 2]
            if x.shape != skip_connection.shape:
                x = F.interpolate(x, size=skip_connection.shape[2:], mode="bilinear", align_corners=True)
            x = self.ups[idx + 1](torch.cat((skip_connection, x), dim=1))

        return self.final_conv(x)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction="none")

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal_loss.mean()

class ComboLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.focal = FocalLoss(alpha=0.75, gamma=2)

    def dice_loss(self, pred, target):
        pred = torch.sigmoid(pred)
        smooth = 1.0
        intersection = (pred * target).sum()
        dice = (2.0 * intersection + smooth) / (pred.sum() + target.sum() + smooth)
        return 1 - dice

    def forward(self, inputs, targets):
        if isinstance(inputs, list):
            loss = 0
            for item in inputs:
                loss += 0.5 * self.focal(item, targets) + 0.5 * self.dice_loss(item, targets)
            return loss / len(inputs)
        return 0.5 * self.focal(inputs, targets) + 0.5 * self.dice_loss(inputs, targets)

# Quick sanity check
_dummy = torch.randn(1, 1, 256, 256)
_model = LightweightUNet()
_out = _model(_dummy)
print(f"Model ok, output shape: {_out.shape}")


In [ ]:
# @title Data prep: build experiment_dataset

def prepare_grand_dataset(
    real_data_dir: str = "real_training_data",
    val_tif_path: str = "val_data/val.tif",
    val_csv_path: str = "val_data/val.csv",
    out_dir: str = "experiment_dataset",
    roi_y: Tuple[int, int] = (ROI_Y_MIN, ROI_Y_MAX),
    roi_x: Tuple[int, int] = (ROI_X_MIN, ROI_X_MAX),
) -> Path:
    out_path = Path(out_dir)
    bonus_images = out_path / "bonus" / "images"
    bonus_masks = out_path / "bonus" / "masks"
    video_images = out_path / "video" / "images"
    video_masks = out_path / "video" / "masks"

    if out_path.exists():
        shutil.rmtree(out_path)
    for p in [bonus_images, bonus_masks, video_images, video_masks]:
        p.mkdir(parents=True, exist_ok=True)

    src_images = Path(real_data_dir) / "images"
    src_masks = Path(real_data_dir) / "masks"
    if not src_images.exists() or not src_masks.exists():
        raise FileNotFoundError("real_training_data must contain images/ and masks/")
    shutil.copytree(src_images, bonus_images, dirs_exist_ok=True)
    shutil.copytree(src_masks, bonus_masks, dirs_exist_ok=True)

    if not Path(val_tif_path).exists():
        raise FileNotFoundError(f"Missing video file: {val_tif_path}")
    if not Path(val_csv_path).exists():
        raise FileNotFoundError(f"Missing CSV file: {val_csv_path}")

    video = tifffile.imread(val_tif_path)
    coords = pd.read_csv(val_csv_path)
    y_min, y_max = roi_y
    x_min, x_max = roi_x
    records: List[dict] = []

    for idx, frame in enumerate(video):
        crop = frame[y_min:y_max, x_min:x_max]
        mask = np.zeros((ROI_H, ROI_W), dtype=np.uint8)
        points = coords[coords["frame"] == idx]
        for _, row in points.iterrows():
            cx = int(round(row["x"] - x_min))
            cy = int(round(row["y"] - y_min))
            if 0 <= cx < ROI_W and 0 <= cy < ROI_H:
                cv2.circle(mask, (cx, cy), 5, 1, -1)
        fname = f"frame_{idx + 1:03}.png"
        cv2.imwrite(str(video_images / fname), crop)
        cv2.imwrite(str(video_masks / fname), mask * 255)
        records.append({"filename": fname, "real_frame_idx": idx})

    pd.DataFrame(records).to_csv(out_path / "video_map.csv", index=False)
    print(f"Dataset written to {out_path.resolve()}")
    print(f"Bonus samples: {len(list(bonus_images.glob('*.png')))} | Video frames: {len(records)}")
    return out_path


In [ ]:
# @title Transforms & Dataset

def get_train_transforms(
    rotate_p: float = 0.7,
    hflip_p: float = 0.5,
    vflip_p: float = 0.5,
    clahe_p: float = 0.5,
    brightness_p: float = 0.5,
    gauss_p: float = 0.3,
    elastic_p: float = 0.2,
    coarse_p: float = 0.5,
    coarse_max_holes: int = 16,
    coarse_min_holes: int = 8,
    coarse_max_hw: int = 16,
    coarse_min_hw: int = 8,
    crop_scale_min: float = 0.8,
    crop_scale_max: float = 1.0,
    crop_ratio_min: float = 0.9,
    crop_ratio_max: float = 1.1,
):
    return A.Compose([
        A.Rotate(limit=180, p=rotate_p),
        A.HorizontalFlip(p=hflip_p),
        A.VerticalFlip(p=vflip_p),
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=clahe_p),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=brightness_p),
        A.GaussNoise(var_limit=(10.0, 50.0), p=gauss_p),
        A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=elastic_p),
        A.CoarseDropout(
            max_holes=coarse_max_holes, max_height=coarse_max_hw, max_width=coarse_max_hw,
            min_holes=coarse_min_holes, min_height=coarse_min_hw, min_width=coarse_min_hw,
            fill_value=0, mask_fill_value=0, p=coarse_p,
        ),
        A.RandomResizedCrop(
            size=(256, 256),
            scale=(crop_scale_min, crop_scale_max),
            ratio=(crop_ratio_min, crop_ratio_max),
            p=0.5,
        ),
        A.PadIfNeeded(min_height=256, min_width=256, border_mode=cv2.BORDER_CONSTANT, value=0),
        ToTensorV2(),
    ])


def get_val_transforms():
    return A.Compose([ToTensorV2()])


class AugmentedMicroscopyDataset(Dataset):
    def __init__(self, root_dir: str, transform=None, return_meta: bool = False):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.return_meta = return_meta
        self.image_paths = sorted(self.root_dir.joinpath("images").glob("*.png"))
        self.mask_paths = sorted(self.root_dir.joinpath("masks").glob("*.png"))
        if len(self.image_paths) != len(self.mask_paths):
            raise ValueError(f"Images and masks counts differ in {root_dir}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx: int):
        img = cv2.imread(str(self.image_paths[idx]), cv2.IMREAD_UNCHANGED)
        mask = cv2.imread(str(self.mask_paths[idx]), cv2.IMREAD_UNCHANGED)
        if img is None or mask is None:
            raise FileNotFoundError(f"Missing pair for index {idx}")
        if img.ndim == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if mask.ndim == 3:
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

        img = img.astype(np.float32)
        mask = (mask > 127).astype(np.float32)

        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img, mask = augmented["image"], augmented["mask"]
        else:
            img = torch.from_numpy(img).float().unsqueeze(0)
            mask = torch.from_numpy(mask).float().unsqueeze(0)

        if isinstance(img, np.ndarray):
            img = torch.from_numpy(img)
        if isinstance(mask, np.ndarray):
            mask = torch.from_numpy(mask)

        img = img.float()
        mask = mask.float()
        if img.max() > 1.0:
            img = img / 255.0
        if img.ndim == 2:
            img = img.unsqueeze(0)
        if mask.ndim == 2:
            mask = mask.unsqueeze(0)

        if self.return_meta:
            return img, mask, {"filename": self.image_paths[idx].name, "dataset_idx": int(idx)}
        return img, mask


In [ ]:
# @title Training, inference, metrics, visualization

def make_fold_loaders(video_root: Path, bonus_root: Path, train_idx, val_idx, batch_size=8, num_workers=2, use_bonus: bool = True, train_transform=None, val_transform=None):
    train_tf = train_transform if train_transform is not None else get_train_transforms()
    val_tf = val_transform if val_transform is not None else get_val_transforms()
    train_ds = Subset(AugmentedMicroscopyDataset(video_root, transform=train_tf), train_idx)
    val_ds = Subset(AugmentedMicroscopyDataset(video_root, transform=val_tf), val_idx)
    datasets = [train_ds]
    bonus_len = 0
    if use_bonus:
        bonus_ds = AugmentedMicroscopyDataset(bonus_root, transform=train_tf)
        datasets.append(bonus_ds)
        bonus_len = len(bonus_ds)
    train_loader = DataLoader(ConcatDataset(datasets), batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=max(1, batch_size // 2), shuffle=False, num_workers=num_workers, pin_memory=True)
    print(f"Train loader: video {len(train_idx)} + bonus {bonus_len} = {len(train_loader.dataset)}")
    print(f"Val loader: {len(val_idx)}")
    return train_loader, val_loader


def train_one_fold(model, train_loader, val_loader, device, epochs=40, lr=1e-3):
    import copy
    criterion = ComboLoss()
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=0.05)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    best_val = float('inf')
    best_state = copy.deepcopy(model.state_dict())
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0.0
        for imgs, masks in train_loader:
            imgs, masks = imgs.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            if isinstance(outputs, (list, tuple)):
                outputs = outputs[-1]
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * imgs.size(0)
        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs, masks = imgs.to(device), masks.to(device)
                outputs = model(imgs)
                if isinstance(outputs, (list, tuple)):
                    outputs = outputs[-1]
                loss = criterion(outputs, masks)
                val_loss += loss.item() * imgs.size(0)
        val_loss /= len(val_loader.dataset)
        scheduler.step(val_loss)
        if val_loss < best_val:
            best_val = val_loss
            best_state = copy.deepcopy(model.state_dict())
        print(f"Epoch {epoch:03d}: train {train_loss:.4f} | val {val_loss:.4f} | best {best_val:.4f}")
    model.load_state_dict(best_state)
    return model, best_val


def infer_fold_oof(model, val_indices, video_root: Path, video_map: Path, device, threshold: float = 0.5):
    infer_ds = Subset(AugmentedMicroscopyDataset(video_root, transform=get_val_transforms(), return_meta=True), val_indices)
    infer_loader = DataLoader(infer_ds, batch_size=1, shuffle=False)
    frame_lookup = pd.read_csv(video_map).set_index('filename')['real_frame_idx'].to_dict()
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs, _, meta in infer_loader:
            filenames = meta['filename'] if isinstance(meta, dict) else [m['filename'] for m in meta]
            imgs = imgs.to(device)
            outputs = model(imgs)
            if isinstance(outputs, (list, tuple)):
                outputs = outputs[-1]
            probs = torch.sigmoid(outputs).cpu().numpy()
            for b in range(probs.shape[0]):
                bin_mask = (probs[b, 0] >= threshold).astype(np.uint8)
                labeled = label(bin_mask)
                for prop in regionprops(labeled):
                    cy, cx = prop.centroid
                    preds.append({'frame': int(frame_lookup[filenames[b]]), 'x': float(cx + ROI_X_MIN), 'y': float(cy + ROI_Y_MIN)})
    return preds


def _filter_roi_frames(df: pd.DataFrame, frames_filter=None, roi=None):
    if frames_filter is not None:
        frame_set = set(map(int, frames_filter))
        df = df[df.frame.isin(frame_set)]
    if roi is not None:
        y_min, y_max, x_min, x_max = roi
        df = df[(df.y >= y_min) & (df.y < y_max) & (df.x >= x_min) & (df.x < x_max)]
    return df


def hota_metric(gt: pd.DataFrame, tr: pd.DataFrame, threshold: float = 5.0):
    from scipy import spatial, optimize
    gt = gt.copy(); tr = tr.copy()
    gt.track_id = gt.track_id.map({old: new for old, new in zip(gt.track_id.unique(), range(gt.track_id.nunique()))})
    tr.track_id = tr.track_id.map({old: new for old, new in zip(tr.track_id.unique(), range(tr.track_id.nunique()))})
    num_gt_ids = gt.track_id.nunique(); num_tr_ids = tr.track_id.nunique()
    frames = sorted(set(gt.frame.unique()) | set(tr.frame.unique()))
    potential_matches_count = np.zeros((num_gt_ids, num_tr_ids))
    gt_id_count = np.zeros((num_gt_ids, 1)); tracker_id_count = np.zeros((1, num_tr_ids))
    HOTA_TP = HOTA_FN = HOTA_FP = 0; LocA = 0.0
    similarities = [1 - np.clip(spatial.distance.cdist(gt[gt.frame == t][['x', 'y']], tr[tr.frame == t][['x', 'y']]) / threshold, 0, 1) for t in frames]
    for t in frames:
        gt_ids_t = gt[gt.frame == t].track_id.to_numpy(); tr_ids_t = tr[tr.frame == t].track_id.to_numpy()
        similarity = similarities[t]
        sim_iou_denom = similarity.sum(0)[np.newaxis, :] + similarity.sum(1)[:, np.newaxis] - similarity
        sim_iou = np.zeros_like(similarity)
        mask = sim_iou_denom > 0 + np.finfo('float').eps
        sim_iou[mask] = similarity[mask] / sim_iou_denom[mask]
        potential_matches_count[gt_ids_t[:, None], tr_ids_t[None, :]] += sim_iou
        gt_id_count[gt_ids_t] += 1; tracker_id_count[0, tr_ids_t] += 1
    global_alignment_score = potential_matches_count / (gt_id_count + tracker_id_count - potential_matches_count)
    matches_count = np.zeros_like(potential_matches_count)
    for t in frames:
        gt_ids_t = gt[gt.frame == t].track_id.to_numpy(); tr_ids_t = tr[tr.frame == t].track_id.to_numpy()
        if len(gt_ids_t) == 0:
            HOTA_FP += len(tr_ids_t); continue
        if len(tr_ids_t) == 0:
            HOTA_FN += len(gt_ids_t); continue
        similarity = similarities[t]
        score_mat = global_alignment_score[gt_ids_t[:, None], tr_ids_t[None, :]] * similarity
        match_rows, match_cols = optimize.linear_sum_assignment(-score_mat)
        matched_mask = similarity[match_rows, match_cols] > 0
        alpha_rows = match_rows[matched_mask]; alpha_cols = match_cols[matched_mask]
        num_matches = len(alpha_rows)
        HOTA_TP += num_matches
        HOTA_FN += len(gt_ids_t) - num_matches
        HOTA_FP += len(tr_ids_t) - num_matches
        if num_matches > 0:
            LocA += sum(similarity[alpha_rows, alpha_cols])
            matches_count[gt_ids_t[alpha_rows], tr_ids_t[alpha_cols]] += 1
    ass_a = matches_count / np.maximum(1, gt_id_count + tracker_id_count - matches_count)
    AssA = np.sum(matches_count * ass_a) / np.maximum(1, HOTA_TP)
    DetA = HOTA_TP / np.maximum(1, HOTA_TP + HOTA_FN + HOTA_FP)
    HOTA = np.sqrt(DetA * AssA)
    return {'HOTA': HOTA, 'AssA': AssA, 'DetA': DetA, 'LocA': LocA, 'HOTA TP': HOTA_TP, 'HOTA FN': HOTA_FN, 'HOTA FP': HOTA_FP}


def link_detections(detections_per_frame, max_dist: float = 7.0):
    # Lightweight nearest-neighbor tracker to assign track_ids to per-frame detections.
    next_id = 0
    active = {}
    records = []
    for frame_idx, dets in enumerate(detections_per_frame):
        assigned = [False] * len(dets)
        new_active = {}
        for tid, (tx, ty, lf) in list(active.items()):
            best = None; best_d = max_dist
            for i, (x, y) in enumerate(dets):
                if assigned[i]:
                    continue
                d = ((x - tx) ** 2 + (y - ty) ** 2) ** 0.5
                if d < best_d:
                    best_d = d; best = i
            if best is not None:
                assigned[best] = True
                new_active[tid] = (dets[best][0], dets[best][1], frame_idx)
                records.append({'frame': frame_idx, 'x': dets[best][0], 'y': dets[best][1], 'track_id': tid})
        for i, (x, y) in enumerate(dets):
            if not assigned[i]:
                tid = next_id; next_id += 1
                new_active[tid] = (x, y, frame_idx)
                records.append({'frame': frame_idx, 'x': x, 'y': y, 'track_id': tid})
        active = new_active
    return pd.DataFrame(records)


def track_detections_simple(preds: pd.DataFrame, max_dist: float):
    max_frame = int(preds.frame.max()) if len(preds) else -1
    dets = [[] for _ in range(max_frame + 1)]
    for _, r in preds.iterrows():
        dets[int(r.frame)].append((float(r.x), float(r.y)))
    tracks_df = link_detections(dets, max_dist=max_dist)
    return tracks_df


def show_val_overlay(model, dataset, val_indices, device: str, threshold: float = 0.5):
    import numpy as np
    if len(val_indices) == 0:
        print("No validation indices to visualize")
        return
    idx = int(np.random.choice(val_indices))
    img, mask = dataset[idx]
    if isinstance(img, tuple):
        img = img[0]
    base = img.squeeze().cpu().numpy()
    if base.max() > 1:
        base = base / 255.0
    with torch.no_grad():
        out = model(img.unsqueeze(0).to(device))
        if isinstance(out, (list, tuple)):
            out = out[-1]
        prob = torch.sigmoid(out)[0, 0].cpu().numpy()
    bin_mask = (prob >= threshold).astype(float)
    gt_mask = mask.squeeze().cpu().numpy()
    gt_props = regionprops(label(gt_mask > 0.5)); pred_props = regionprops(label(bin_mask))
    gx, gy = [], []
    for p in gt_props:
        cy, cx = p.centroid; gx.append(cx); gy.append(cy)
    px, py = [], []
    for p in pred_props:
        cy, cx = p.centroid; px.append(cx); py.append(cy)
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes[0].imshow(base, cmap='gray'); axes[0].set_title(f"Val image (idx {idx})")
    axes[1].imshow(prob, cmap='viridis'); axes[1].set_title("Prob map")
    axes[2].imshow(base, cmap='gray')
    axes[2].scatter(gx, gy, s=35, facecolors='none', edgecolors='lime', linewidths=1.2, label='GT')
    axes[2].scatter(px, py, s=30, marker='x', color='red', linewidths=1.2, label='Pred')
    axes[2].set_title("Overlay: red=pred (x), green=gt (o)")
    axes[2].legend(loc='upper right')
    for ax in axes:
        ax.axis('off')
    plt.tight_layout(); plt.show()


In [ ]:
# @title BTrack integration + orchestration

def run_btrack_tracking(oof_csv: str, output_csv: str = "tracks_btrack.csv", config_path: str | None = None, max_search_radius: float = 12.0):
    try:
        import btrack
    except ImportError as e:
        raise ImportError("btrack is not installed. Install with `pip install btrack` or rerun the setup cell.") from e

    df = pd.read_csv(oof_csv)
    detections = [{"t": int(row.frame), "x": float(row.x), "y": float(row.y), "z": 0.0, "r": 5.0} for _, row in df.iterrows()]
    if config_path is None:
        config_path = btrack.datasets.cell_config()

    with btrack.BayesianTracker() as tracker:
        tracker.configure_from_file(config_path)
        tracker.max_search_radius = max_search_radius
        tracker.append(detections)
        tracker.volume = ((0, 1024), (0, 1024), (0, 1))
        tracker.track_interactive(step_size=100)
        tracker.optimize()
        tracks_out = []
        for track in tracker.tracks:
            for node in track:
                tracks_out.append({"track_id": int(track.ID), "frame": int(node.t), "x": float(node.x), "y": float(node.y)})
    tracks_df = pd.DataFrame(tracks_out).sort_values(["frame", "track_id"])
    tracks_df.to_csv(output_csv, index=False)
    print(f"BTrack tracks saved to {output_csv}")
    return tracks_df


def run_grand_kfold(
    out_dir: str = "experiment_dataset",
    k_splits: int = 5,
    use_bonus: bool = True,
    epochs: int = 30,
    batch_size: int = 8,
    lr: float = 1e-3,
    num_workers: int = 2,
    threshold: float = 0.5,
    save_dir: str = ".",
    visualize_samples: bool = True,
    sample_idx: int = 0,
    match_thresh: float = 5.0,
    run_btrack: bool = False,
    btrack_config: str | None = None,
    btrack_radius: float = 12.0,
    train_aug_kwargs: dict | None = None,
):
    out_path = Path(out_dir)
    video_root = out_path / "video"
    bonus_root = out_path / "bonus"
    video_map = out_path / "video_map.csv"

    train_tf = get_train_transforms(**(train_aug_kwargs or {}))
    val_tf = get_val_transforms()

    video_base = AugmentedMicroscopyDataset(video_root, transform=None)
    bonus_base = AugmentedMicroscopyDataset(bonus_root, transform=train_tf)
    n_frames = len(video_base)
    if n_frames == 0:
        raise ValueError("No frames found in experiment_dataset/video. Run prepare_grand_dataset first.")

    print(f"Frames: {n_frames}, Bonus: {len(bonus_base)} (use_bonus={use_bonus})")

    kf = KFold(n_splits=k_splits, shuffle=True, random_state=42)
    all_preds = []
    model_paths = []
    fold_detAs = []
    fold_hota_detAs = []

    roi_tuple = (ROI_Y_MIN, ROI_Y_MAX, ROI_X_MIN, ROI_X_MAX)

    for fold, (train_idx, val_idx) in enumerate(kf.split(range(n_frames)), start=1):
        print(f"
========== Fold {fold} / {k_splits} ==========")
        train_loader, val_loader = make_fold_loaders(
            video_root, bonus_root, train_idx, val_idx,
            batch_size=batch_size, num_workers=num_workers, use_bonus=use_bonus,
            train_transform=train_tf, val_transform=val_tf,
        )
        model = LightweightUNet(in_channels=1, n_classes=1).to(device)
        model, best_val = train_one_fold(model, train_loader, val_loader, device, epochs=epochs, lr=lr)

        ckpt_path = Path(save_dir) / f"model_fold_{fold}.pth"
        torch.save(model.state_dict(), ckpt_path)
        model_paths.append(ckpt_path)
        print(f"Saved best checkpoint (val_loss={best_val:.4f}) to {ckpt_path}")

        fold_preds = infer_fold_oof(
            model,
            val_idx,
            video_root=video_root,
            video_map=video_map,
            device=device,
            threshold=threshold,
        )
        fold_df = pd.DataFrame(fold_preds)
        all_preds.extend(fold_preds)

        # Track detections per fold and compute HOTA-style DetA
        tracks_df_fold = track_detections_simple(fold_df, max_dist=match_thresh)
        gt_fold = _filter_roi_frames(pd.read_csv("val_data/val.csv"), frames_filter=val_idx, roi=roi_tuple)
        tr_fold = _filter_roi_frames(tracks_df_fold.rename(columns={'track_id': 'track_id'}), frames_filter=val_idx, roi=roi_tuple)
        hota_res_fold = hota_metric(gt_fold, tr_fold, threshold=match_thresh)
        fold_hota_detAs.append(hota_res_fold['DetA'])
        print(f"Fold {fold} HOTA DetA={hota_res_fold['DetA']:.4f} (TP={hota_res_fold['HOTA TP']} FP={hota_res_fold['HOTA FP']} FN={hota_res_fold['HOTA FN']})")

        if visualize_samples and len(val_idx) > 0:
            val_ds = AugmentedMicroscopyDataset(video_root, transform=val_tf)
            show_val_overlay(model, val_ds, val_indices=val_idx, device=device, threshold=threshold)

    oof_df = pd.DataFrame(all_preds).sort_values("frame").reset_index(drop=True)
    oof_path = Path(save_dir) / "oof_predictions.csv"
    oof_df.to_csv(oof_path, index=False)
    print(f"OOF predictions saved to {oof_path}")

    # Track full OOF detections and compute HOTA-style DetA
    tracks_df = track_detections_simple(oof_df, max_dist=match_thresh)
    gt_full = _filter_roi_frames(pd.read_csv("val_data/val.csv"), frames_filter=None, roi=roi_tuple)
    tr_full = _filter_roi_frames(tracks_df, frames_filter=None, roi=roi_tuple)
    hota_res_full = hota_metric(gt_full, tr_full, threshold=match_thresh)
    print(f"HOTA DetA (tracked OOF) @ {match_thresh}px: {hota_res_full['DetA']:.4f} (TP={hota_res_full['HOTA TP']} FP={hota_res_full['HOTA FP']} FN={hota_res_full['HOTA FN']})")
    if fold_hota_detAs:
        print(f"Average fold HOTA DetA: {np.mean(fold_hota_detAs):.4f}")

    if run_btrack:
        tracks_df = run_btrack_tracking(
            str(oof_path),
            output_csv=str(Path(save_dir) / "tracks_btrack.csv"),
            config_path=btrack_config,
            max_search_radius=btrack_radius,
        )

    if visualize_samples:
        sample_ds = AugmentedMicroscopyDataset(video_root, transform=val_tf)
        visualize_fold_predictions(model_paths, sample_ds, device=device, threshold=threshold, sample_idx=sample_idx)

    return oof_df, model_paths, tracks_df


print("Helpers ready: prepare_grand_dataset, run_grand_kfold, run_btrack_tracking")


In [ ]:
# @title Download / Fetch Data (run once)
AUTO_FETCH_DRIVE = False  # set True to copy training data from Drive
DRIVE_SOURCE_PATH = "/content/drive/MyDrive/unet_train"
TARGET_DIR = "real_training_data"

# Download validation data if missing
if not (Path("val_data/val.tif").exists() and Path("val_data/val.csv").exists()):
    download_validation_data(target_dir="val_data")
else:
    print("Validation data already present.")

# Optional Drive copy for training data
if AUTO_FETCH_DRIVE:
    fetch_from_drive(DRIVE_SOURCE_PATH=DRIVE_SOURCE_PATH, TARGET_DIR=TARGET_DIR)
else:
    print("Skipping Drive fetch (AUTO_FETCH_DRIVE=False). Ensure real_training_data/ exists with images/ and masks/.")


In [ ]:
# @title Run Full Pipeline (edit params as needed)
# Paths
REAL_DATA_DIR = "real_training_data"
VAL_TIF = "val_data/val.tif"
VAL_CSV = "val_data/val.csv"
OUT_DIR = "experiment_dataset"
SAVE_DIR = "."

# Pipeline knobs
K_SPLITS = 5
USE_BONUS = True          # set False to ignore bonus set
EPOCHS = 30               # keep modest for T4
BATCH_SIZE = 8
LR = 1e-3
THRESHOLD = 0.5           # sigmoid->mask threshold for detections
MATCH_THRESH = 5.0        # px for DetA calculation
VISUALIZE = True          # show sample masks from all folds
SAMPLE_IDX = 0            # which sample to visualize
RUN_BTRACK = True         # toggle btrack tracking
BTRACK_CONFIG = None      # None -> use btrack default cell config
BTRACK_RADIUS = 12.0      # max_search_radius
USE_TRAIN_AUG = True      # set False to disable train-time augmentations

# Augmentation knobs
TRAIN_AUG_KWARGS = dict(
    rotate_p=0.7,
    hflip_p=0.5,
    vflip_p=0.5,
    clahe_p=0.5,
    brightness_p=0.5,
    gauss_p=0.3,
    elastic_p=0.2,
    coarse_p=0.5,
    coarse_max_holes=16,
    coarse_min_holes=8,
    coarse_max_hw=16,
    coarse_min_hw=8,
    crop_scale_min=0.8,
    crop_scale_max=1.0,
    crop_ratio_min=0.9,
    crop_ratio_max=1.1,
)

# 1) Build grand dataset (cleans OUT_DIR each time)
prepare_grand_dataset(
    real_data_dir=REAL_DATA_DIR,
    val_tif_path=VAL_TIF,
    val_csv_path=VAL_CSV,
    out_dir=OUT_DIR,
)

# 2) Train K-fold, infer OOF, compute DetA, (optionally) run btrack, visualize
oof_df, model_paths, tracks_df = run_grand_kfold(
    out_dir=OUT_DIR,
    k_splits=K_SPLITS,
    use_bonus=USE_BONUS,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    lr=LR,
    threshold=THRESHOLD,
    save_dir=SAVE_DIR,
    visualize_samples=VISUALIZE,
    sample_idx=SAMPLE_IDX,
    match_thresh=MATCH_THRESH,
    run_btrack=RUN_BTRACK,
    btrack_config=BTRACK_CONFIG,
    btrack_radius=BTRACK_RADIUS,
    train_aug_kwargs=TRAIN_AUG_KWARGS if USE_TRAIN_AUG else {},
)

print("Done. OOF detections:", len(oof_df), "| Models:", model_paths)
if tracks_df is not None:
    print("BTrack tracks:", len(tracks_df))
